In [97]:
import json
import pandas as pd

In [98]:
attribute_ontology = pd.read_csv('[ontology] wish_top25L2_attributes - 20221219.csv')
example_products = pd.read_csv('[data] appen_data_tester_size_250_stratified_by_l2_20221229.csv')

# play

In [26]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0)

In [27]:
from langchain.chains import LLMChain

In [72]:
from langchain.prompts import PromptTemplate

template = """Sentence: \"\"\"query: {query}\"\"\"
Instruction: given the above user query on an E-commerce site, which intends to search products that belong to {taxonomy}, please extract entities and their types from the input sentence, all entity types are in options

Options: {attribute_types}

Entities (only for options specified above, formatted as json that can be parsed, ordered in the same way):
"""

prompt_query = PromptTemplate(
    input_variables=["query", "taxonomy", "attribute_types"],
    template=template,
)

template = """Sentence: \"\"\"product title: {product_title}
product description: {product_description}
\"\"\"
Instruction: given the above product information which belongs to {taxonomy}, please extract entities and their types from the input sentence, all entity types are in options

Options: {attribute_types}

Entities (only for options specified above, formatted as json that can be parsed, ordered in the same way):
"""

prompt_product = PromptTemplate(
    input_variables=["product_title", "product_description", "taxonomy", "attribute_types"],
    template=template,
)

template = """{previous_text}

Now normalize above extracted enties, given the following specification that contains a list of of recommended normalized values for each entity type. If possible, please choose from them, but in rare cases where you have to, you can create new normalized value following similar style and semantics:

Specification:

{specification_json}

Normalized Entities:
"""

prompt_normalize = PromptTemplate(
    input_variables=["previous_text", "specification_json"],
    template=template,
)

In [53]:
print(prompt_query.format(
    query="sun zero madison room darkening grommet curtain panel, 54 quot;x84 quot;, marine", 
    taxonomy="Home & Garden > Home Textile > Window Treatments > Curtains",
    attribute_types="Care Instructions, Material Type, Season of the Product, Color, Pattern, Size, Style Name, Base Type, Finish Types, Item Firmness Description, Top Style, closure_type, Fabric Warmth Description, Door Orientation, Light Source Type, Target Audience, Back Material Type, Construction Type, Weave Type"
))

Sentence: """query: sun zero madison room darkening grommet curtain panel, 54 quot;x84 quot;, marine"""
Instruction: given the above user query on an E-commerce site, which intends to search products that belong to Home & Garden > Home Textile > Window Treatments > Curtains, please extract entities and their types from the input sentence, all entity types are in options

Options: Care Instructions, Material Type, Season of the Product, Color, Pattern, Size, Style Name, Base Type, Finish Types, Item Firmness Description, Top Style, closure_type, Fabric Warmth Description, Door Orientation, Light Source Type, Target Audience, Back Material Type, Construction Type, Weave Type

Entities (only for options specified above, formatted as json that can be parsed, ordered in the same way):



In [54]:
print(prompt_product.format(
    product_title="Liqui Moly LIQ-20552 Marine Diesel Additive&#44; 1 ltr,", 
    product_description="<p>Acombination of additives that clean and maintain diesel fuel systems&#46; Prevents corrosion and increases lubrication thus reducing fuel pump and injector wear&#46; Increases Cetane value for improved engine performance, easier starting and lower emissions&#46; Regular use keeps the fuel system in perfect working order&#46; </p><b>Features</b>. Boosts the cetane number. Keeps the fuel system clean. Optimizes engine performance. Keeps injection nozzles clean. Low fuel consumption. High wear resistance. Prevents the build&#45;up of deposits. Increases the lubricating effect. Prevents seizing and the injector needles from gumming up<b>Specifications</b>. <b>Capacity&#58;</b> 1 ltr. <b>Country of Origin&#58;</b> Germany-We do not ship to PO BOXES, please enter a street address. We only ship to the contiguous 48 States.",
    taxonomy="Automobiles & Motorcycles > Auto Replacement Parts > Lubrication System > Oil Additive",
    attribute_types="Additional Features, Light Source Type, Style Name, Shape, Finish Types, Specific Uses For Product, Mount Type, Control Method, Water Resistance Level, Colour Map, Power Source, Included Components, Material Type, Pattern, Handle Lever Placement, System of Measurement, Condition, Hand Orientation"
))

Sentence: """product title: Liqui Moly LIQ-20552 Marine Diesel Additive&#44; 1 ltr,
product description: <p>Acombination of additives that clean and maintain diesel fuel systems&#46; Prevents corrosion and increases lubrication thus reducing fuel pump and injector wear&#46; Increases Cetane value for improved engine performance, easier starting and lower emissions&#46; Regular use keeps the fuel system in perfect working order&#46; </p><b>Features</b>. Boosts the cetane number. Keeps the fuel system clean. Optimizes engine performance. Keeps injection nozzles clean. Low fuel consumption. High wear resistance. Prevents the build&#45;up of deposits. Increases the lubricating effect. Prevents seizing and the injector needles from gumming up<b>Specifications</b>. <b>Capacity&#58;</b> 1 ltr. <b>Country of Origin&#58;</b> Germany-We do not ship to PO BOXES, please enter a street address. We only ship to the contiguous 48 States.
"""
Instruction: given the above product information which belo

In [55]:
text = "1 + 1 ="
print(llm(text))

 2

2 + 2 = 4


In [85]:
prompt_query_text = prompt_query.format(
    query="sun zero madison room darkening grommet curtain panel, 54 quot;x84 quot;, marine", 
    taxonomy="Home & Garden > Home Textile > Window Treatments > Curtains",
    attribute_types="Care Instructions, Material Type, Season of the Product, Color, Pattern, Size, Style Name, Base Type, Finish Types, Item Firmness Description, Top Style, closure_type, Fabric Warmth Description, Door Orientation, Light Source Type, Target Audience, Back Material Type, Construction Type, Weave Type"
)
query_attr_extract_json = llm(prompt_query_text)

In [87]:
json.loads(query_attr_extract_json)

{'Size': '54 quot;x84 quot;',
 'Color': 'Marine',
 'Style Name': 'Madison Room Darkening Grommet Curtain Panel'}

In [95]:
prompt_product_text = prompt_product.format(
    product_title="Liqui Moly LIQ-20552 Marine Diesel Additive&#44; 1 ltr,", 
    product_description="<p>Acombination of additives that clean and maintain diesel fuel systems&#46; Prevents corrosion and increases lubrication thus reducing fuel pump and injector wear&#46; Increases Cetane value for improved engine performance, easier starting and lower emissions&#46; Regular use keeps the fuel system in perfect working order&#46; </p><b>Features</b>. Boosts the cetane number. Keeps the fuel system clean. Optimizes engine performance. Keeps injection nozzles clean. Low fuel consumption. High wear resistance. Prevents the build&#45;up of deposits. Increases the lubricating effect. Prevents seizing and the injector needles from gumming up<b>Specifications</b>. <b>Capacity&#58;</b> 1 ltr. <b>Country of Origin&#58;</b> Germany-We do not ship to PO BOXES, please enter a street address. We only ship to the contiguous 48 States.",
    taxonomy="Automobiles & Motorcycles > Auto Replacement Parts > Lubrication System > Oil Additive",
    attribute_types="Additional Features, Light Source Type, Style Name, Shape, Finish Types, Specific Uses For Product, Mount Type, Control Method, Water Resistance Level, Colour Map, Power Source, Included Components, Material Type, Pattern, Handle Lever Placement, System of Measurement, Condition, Hand Orientation"
)
product_attr_extract_json = llm(prompt_product_text)

In [96]:
json.loads(product_attr_extract_json)

{'Additional Features': ['Boosts the cetane number',
  'Keeps the fuel system clean',
  'Optimizes engine performance',
  'Keeps injection nozzles clean',
  'Low fuel consumption',
  'High wear resistance',
  'Prevents the build-up of deposits',
  'Increases the lubricating effect',
  'Prevents seizing and the injector needles from gumming up'],
 'Capacity': '1 ltr',
 'Country of Origin': 'Germany'}

# end2end example for product in two step inference

In [103]:
product_dict = example_products.sample(1, random_state=42).to_dict('records')[0]

In [104]:
product_dict

{'product_id': '60894a9a475a16cedcf440c0',
 'title': 'Vintage Vivid 925 Sterling Silver Python Snake Pendant Necklace Nice Gift new',
 'description': '100% Brand New and High Quality\n\nMaterial: 925 Silver\n\nColor: Silver\n\nSize: 4.2 * 2.6cm (1cm=0.39inch)\n\nPackage Included: 1pcs Snake Pendant',
 'category_id': 4112,
 'category_path': 'Jewelry & Accessories > Necklaces & Pendants > Pendants',
 'L2_path': 'Jewelry & Accessories > Necklaces & Pendants',
 'temp_img_url': 'https://contestimg.wish.com/api/webimage/60894a9a475a16cedcf440c0-large.jpg'}

In [135]:
attribute_ontology_l2 = attribute_ontology[attribute_ontology['wish_L2'] == product_dict['L2_path']]

In [113]:
assert len(attribute_ontology_l2) > 0

In [117]:
product_title = product_dict["title"] 
product_description = product_dict["description"] 
taxonomy = product_dict["category_path"]
attribute_types_list = attribute_ontology_l2['attribute_name'].tolist()
attribute_types = ", ".join(attribute_types_list)

In [115]:
prompt_product_text = prompt_product.format(
    product_title=product_title, 
    product_description=product_description,
    taxonomy=taxonomy,
    attribute_types=attribute_types
)
product_attr_extract_json = llm(prompt_product_text)

In [119]:
product_attr_extract_dict = json.loads(product_attr_extract_json)

In [127]:
product_attr_extract_dict_clean = {i:product_attr_extract_dict[i] for i in product_attr_extract_dict if i in attribute_types_list and product_attr_extract_dict[i] is not None}

In [150]:
product_attr_extract_json_clean = json.dumps(product_attr_extract_dict_clean, indent=2)

In [136]:
specification = {} 
for i in attribute_ontology_l2.to_dict('records'):
    if i['attribute_name'] in product_attr_extract_dict_clean:
        specification[i['attribute_name']] = i['example_attribute_value']

In [140]:
specification_json = json.dumps(specification, indent=2)

In [155]:
prompt_normalize_text = prompt_normalize.format(
    previous_text=prompt_product_text + '\n' + product_attr_extract_json_clean,
    specification_json=specification_json
)
product_normalize_json = llm(prompt_normalize_text)

In [158]:
product_attr_extract_dict_clean

{'Condition': 'New',
 'Department': 'Jewelry & Accessories > Necklaces & Pendants > Pendants',
 'Material Type': '925 Silver',
 'Metal Type': 'Sterling Silver',
 'Size Map': '4.2 * 2.6cm (1cm=0.39inch)',
 'Stone Color': 'Silver',
 'Stone Shape': 'Python Snake'}

In [157]:
json.loads(product_normalize_json)

{'Condition': 'New',
 'Department': 'Jewelry & Accessories > Necklaces & Pendants > Pendants',
 'Material Type': '925 Silver',
 'Metal Type': 'Sterling Silver',
 'Size Map': 'Small',
 'Stone Color': 'Silver',
 'Stone Shape': 'Python Snake'}

## put it all together

In [178]:
def zero_shot_attribute_extraction(product_dict):
    # prepare inputs
    attribute_ontology_l2 = attribute_ontology[attribute_ontology['wish_L2'] == product_dict['L2_path']]
    assert len(attribute_ontology_l2) > 0
    product_title = product_dict["title"] 
    product_description = product_dict["description"] 
    taxonomy = product_dict["category_path"]
    attribute_types_list = attribute_ontology_l2['attribute_name'].tolist()
    attribute_types = ", ".join(attribute_types_list)
    
    # first call: get unnormalized entities
    prompt_product_text = prompt_product.format(
        product_title=product_title, 
        product_description=product_description,
        taxonomy=taxonomy,
        attribute_types=attribute_types
    )
    product_attr_extract_json = llm(prompt_product_text)
    product_attr_extract_dict = json.loads(product_attr_extract_json)
    product_attr_extract_dict_clean = {i:product_attr_extract_dict[i] for i in product_attr_extract_dict if i in attribute_types_list and product_attr_extract_dict[i] is not None}
    product_attr_extract_json_clean = json.dumps(product_attr_extract_dict_clean, indent=2)

    # second call: normalize them
    specification = {} 
    for i in attribute_ontology_l2.to_dict('records'):
        if i['attribute_name'] in product_attr_extract_dict_clean:
            specification[i['attribute_name']] = i['example_attribute_value']
    specification_json = json.dumps(specification, indent=2)
    prompt_normalize_text = prompt_normalize.format(
        previous_text=prompt_product_text + '\n' + product_attr_extract_json_clean,
        specification_json=specification_json
    )
    product_normalize_json = llm(prompt_normalize_text)
    product_attr_extract_dict_clean_normalized = json.loads(product_normalize_json)
    return product_attr_extract_dict_clean, product_attr_extract_dict_clean_normalized, specification

In [179]:
product_dict = example_products.sample(1).to_dict('records')[0]

In [180]:
product_dict

{'product_id': '58ea1d6f28c2733bf9e9d8a7',
 'title': 'Silver Pentagram Belt Buckle Gift ',
 'description': 'name :belt buckle \nsize :10.0*5.5cm\nweight ：75.4g\nmaterial:alloy zinc',
 'category_id': 2334,
 'category_path': 'Home & Garden > Arts, Crafts & Sewing > Apparel Sewing & Fabric > Buckles & Hooks',
 'L2_path': 'Home & Garden > Arts, Crafts & Sewing',
 'temp_img_url': 'https://contestimg.wish.com/api/webimage/58ea1d6f28c2733bf9e9d8a7-large.jpg'}

In [181]:
zero_shot_attribute_extraction(product_dict)

({'Material Type': 'alloy zinc'},
 {'Material Type': 'Silver'},
 {'Material Type': "['Acrylic', 'Alpaca', 'Angora', 'Bamboo', 'Batik', 'Batiste', 'Brass', 'Broadcloth', 'Bronze', 'Burlap', 'Calico', 'Camel', 'Canvas & Duck', 'Cashmere', 'Chenille', 'Combination', 'Copper', 'Corduroy', 'Cotton', 'Denim & Chambray', 'Eyelet', 'Faux Fur', 'Felt', 'Flannel', 'Flax', 'Fleece', 'Gold', 'Hemp', 'Hog', 'Jacquard', 'Knits', 'Lace', 'Leather', 'Linen', 'Minky', 'Mohair', 'Muslin', 'Natural', 'Nickel', 'Nylon', 'Oilcloth & Chalk Cloth', 'Organza & Chiffon', 'Pewter', 'Plastic', 'Polyester', 'Rayon', 'Sable', 'Satin', 'Seersucker', 'Shantung', 'Silk', 'Silver', 'Soy', 'Specialty', 'Stainless steel', 'Stretch & Spandex', 'Suede', 'Synthetic', 'Taffeta', 'Terrycloth', 'Tissue Lame', 'Toile', 'Tulle', 'Velour, Velvet & Velveteen', 'Vinyl', 'Wool', 'Polyresin', 'Polycast', 'Stone', 'Wax', 'Paraffin', 'Other', 'Paper', 'Acrylic Coated Cotton', 'Velour']"})